<a href="https://colab.research.google.com/github/abdullahyasser0/RAG-Langchain-/blob/main/RAG_langchain_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#what is langchain ?

---


LangChain is an open-source framework that simplifies the development of applications powered by large language models (LLMs).

#imports

In [7]:
!pip install langchain faiss-cpu transformers
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25


In [3]:
!wget -O arxiv_abs_title.zip "https://zenodo.org/records/3496527/files/gcunhase%2FArXivAbsTitleDataset-v1.0.zip?download=1"
!unzip arxiv_abs_title.zip

--2025-06-21 15:16:25--  https://zenodo.org/records/3496527/files/gcunhase%2FArXivAbsTitleDataset-v1.0.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13098401 (12M) [application/octet-stream]
Saving to: ‘arxiv_abs_title.zip’

arxiv_abs_title.zip 100%[===================>]  12.49M   547KB/s    in 22s     

2025-06-21 15:16:48 (581 KB/s) - ‘arxiv_abs_title.zip’ saved [13098401/13098401]

Archive:  arxiv_abs_title.zip
923c122438b1317f9db090a5858aecb7c9d344f8
replace gcunhase-ArXivAbsTitleDataset-923c122/.gitignore? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: gcunhase-ArXivAbsTitleDataset-923c122/.gitignore  
replace gcunhase-ArXivAbsTitleDataset-923c122/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: gcunhase-ArXivAbsTitleDataset-923c122/LICENSE  
replace gcunhase-ArXivAbsTitleDataset-923

# Load and Chunk Your Documents

In [5]:
from langchain_community.document_loaders import TextLoader

files = [
    '/content/gcunhase-ArXivAbsTitleDataset-923c122/results/artificial intelligence_10047_15000_15_abs.txt',
    '/content/gcunhase-ArXivAbsTitleDataset-923c122/results/artificial intelligence_10047_15000_15_title.txt'
]

# Load documents from each file
docs = []
for file in files:
    loader = TextLoader(file)
    docs.extend(loader.load())


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

#Generate Embeddings

Please add your HF_TOKEN token to Colab secrets (Huggingface token) [How to generate?](https://huggingface.co/docs/hub/en/security-tokens)

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

# Use sentence-transformers for generating embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedding_model.embed_documents([chunk.page_content for chunk in chunks])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#Create the Vector Store Using FAISS

---
FAISS, short for Facebook AI Similarity Search, is an open-source library focused on efficient similarity search and clustering of dense vectors. It allows users to index and search through large collections of vectors with speed and memory efficiency.


In [14]:
from langchain.vectorstores import FAISS

# Create a FAISS vector store from document embeddings
vectorstore = FAISS.from_documents(documents=chunks, embedding=embedding_model)

# Build the RAG Pipeline

In [21]:
retriever = vectorstore.as_retriever()

In [22]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

local_pipe = pipeline("text2text-generation", model="google/flan-t5-base", device=0)  # use device=0 for GPU

llm = HuggingFacePipeline(pipeline=local_pipe)

Device set to use cuda:0


what dose the following cell do ?

---



1.    Initializes a prompt template
2.    Builds a document combination strategy (stuff, map_reduce, etc.)
3.    Connects it to the retriever + LLM
4.    Returns a ready-to-run Retrieval-Augmented Generation pipeline









In [28]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"  # Stuff all retrieved docs into a single prompt.
)



| Chain Type     | Description                                                                              |
| -------------- | ---------------------------------------------------------------------------------------- |
| `"stuff"`      | Stuff all retrieved docs into a single prompt. Fast, simple, but may hit token limits.   |
| `"map_reduce"` | Run LLM on each doc, then combine results. Better for longer texts.                      |
| `"refine"`     | Iteratively improve an answer by passing documents one by one. More accurate but slower. |


In [27]:
question = "can a modern email client can proactively retrieve relevant attachable items from the user\'s past emails based on the context of the current conversation?"
response = rag_pipeline.run(question)
print(f"Q: {question}\nA: {response}")

Q: can a modern email client can proactively retrieve relevant attachable items from the user's past emails based on the context of the current conversation?
A: can a modern email client proactively retrieve relevant attachable items from the user's past emails based on the context of the current past emails based on the context of the current conversation, and recommend them for inclusion, to reduce the time and effort involved in composing the response


RAG opens up exciting possibilities in areas like chatbots, customer support, and information retrieval. Start experimenting with your own datasets and see the difference RAG can make in your projects!

[Refrance](https://medium.com/@mehar.chand.cloud/how-to-build-a-simple-retrieval-augmented-generation-rag-system-f6ffaf8a705c)